
# 🧠 Przykłady komunikacji sieciowej w Pythonie (TCP i UDP)

Ten notatnik zawiera przykłady implementacji prostych serwerów i klientów sieciowych w języku **Python**,
z wykorzystaniem protokołów **TCP** i **UDP**.

Każdy fragment kodu można uruchomić niezależnie w środowisku Jupyter Notebook lub jako osobny plik `.py`.


## 1️⃣ Serwer TCP — prosty (jednowątkowy)

In [ ]:

import socket

HOST = '127.0.0.1'
PORT = 5000

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((HOST, PORT))
server_socket.listen()

print(f"Serwer TCP działa na {HOST}:{PORT}")

conn, addr = server_socket.accept()
print(f"Połączono z {addr}")

with conn:
    while True:
        data = conn.recv(1024)
        if not data:
            break
        print(f"Odebrano: {data.decode()}")
        conn.sendall(data)

print("Połączenie zakończone.")


## 2️⃣ Serwer TCP — wielowątkowy z obsługą komend

In [ ]:

import socket
import threading
import datetime
import random

HOST = '127.0.0.1'
PORT = 5000

def handle_client(conn, addr):
    print(f"[NOWE POŁĄCZENIE] {addr}")
    with conn:
        while True:
            data = conn.recv(1024)
            if not data:
                break
            message = data.decode().strip()
            if not message:
                continue
            print(f"[{addr}] {message}")
            parts = message.split(maxsplit=1)
            command = parts[0].lower()
            arg = parts[1] if len(parts) > 1 else ""

            if command == "echo":
                response = arg
            elif command == "time":
                response = datetime.datetime.now().strftime("%H:%M:%S")
            elif command == "date":
                response = datetime.date.today().strftime("%Y-%m-%d")
            elif command == "random":
                response = str(random.randint(1, 1000))
            else:
                response = f"Nieznane polecenie: {command}"

            conn.sendall((response + "\n").encode())
    print(f"[ROZŁĄCZONO] {addr}")

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server.bind((HOST, PORT))
server.listen()
print(f"[START] Serwer TCP działa na {HOST}:{PORT}")

while True:
    conn, addr = server.accept()
    thread = threading.Thread(target=handle_client, args=(conn, addr), daemon=True)
    thread.start()
    print(f"[AKTYWNE POŁĄCZENIA] {threading.active_count() - 1}")


## 3️⃣ Serwer TCP — asynchroniczny (`asyncio`)

In [ ]:

import asyncio
import datetime
import random

HOST = '127.0.0.1'
PORT = 5000

async def handle_client(reader: asyncio.StreamReader, writer: asyncio.StreamWriter):
    addr = writer.get_extra_info('peername')
    print(f"[NOWE POŁĄCZENIE] {addr}")

    while True:
        data = await reader.read(1024)
        if not data:
            break
        message = data.decode().strip()
        print(f"[{addr}] {message}")
        parts = message.split(maxsplit=1)
        cmd = parts[0].lower()
        arg = parts[1] if len(parts) > 1 else ""

        if cmd == "echo":
            response = arg
        elif cmd == "time":
            response = datetime.datetime.now().strftime("%H:%M:%S")
        elif cmd == "date":
            response = datetime.date.today().strftime("%Y-%m-%d")
        elif cmd == "random":
            response = str(random.randint(1, 1000))
        else:
            response = f"Nieznane polecenie: {cmd}"

        writer.write((response + "\n").encode())
        await writer.drain()

    print(f"[ROZŁĄCZONO] {addr}")
    writer.close()
    await writer.wait_closed()

async def main():
    server = await asyncio.start_server(handle_client, HOST, PORT)
    addr = server.sockets[0].getsockname()
    print(f"[START] Serwer TCP (asyncio) działa na {addr}")

    async with server:
        await server.serve_forever()

if __name__ == "__main__":
    asyncio.run(main())


## 4️⃣ Klient TCP — interaktywny

In [ ]:

import socket

HOST = '127.0.0.1'
PORT = 5000

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client:
    client.connect((HOST, PORT))
    print(f"Połączono z serwerem {HOST}:{PORT}")
    print("Wpisz komendę (echo, time, date, random) lub 'exit' aby zakończyć.")

    while True:
        message = input("> ")
        if message.lower() in ("exit", "quit"):
            print("Zakończono połączenie.")
            break

        client.sendall((message + "\n").encode())
        data = client.recv(1024)
        if not data:
            print("Serwer zakończył połączenie.")
            break
        print("Odpowiedź:", data.decode().strip())


## 5️⃣ Serwer UDP — odbiera wiadomości

In [ ]:

import socket

HOST = "127.0.0.1"
PORT = 6000

with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
    s.bind((HOST, PORT))
    print(f"[START] Serwer UDP nasłuchuje na {HOST}:{PORT}")
    while True:
        data, addr = s.recvfrom(1024)
        print(f"Otrzymano od {addr}: {data.decode()}")


## 6️⃣ Klient UDP — wysyła wiadomości

In [ ]:

import socket

HOST = "127.0.0.1"
PORT = 6000

with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
    print(f"Połączono z {HOST}:{PORT} (UDP)")
    print("Wpisz wiadomość i naciśnij Enter. 'exit' kończy działanie.")

    while True:
        message = input("> ")
        if message.lower() in ("exit", "quit"):
            break
        s.sendto(message.encode(), (HOST, PORT))


## 7️⃣ Serwer UDP — odpowiada klientowi

In [ ]:

import socket
import datetime
import random

HOST = "127.0.0.1"
PORT = 6000

with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
    s.bind((HOST, PORT))
    print(f"[START] Serwer UDP działa na {HOST}:{PORT}")
    while True:
        data, addr = s.recvfrom(1024)
        message = data.decode().strip()
        print(f"[{addr}] -> {message}")
        parts = message.split(maxsplit=1)
        cmd = parts[0].lower()
        arg = parts[1] if len(parts) > 1 else ""

        if cmd == "echo":
            response = arg
        elif cmd == "time":
            response = datetime.datetime.now().strftime("%H:%M:%S")
        elif cmd == "date":
            response = datetime.date.today().strftime("%Y-%m-%d")
        elif cmd == "random":
            response = str(random.randint(1, 1000))
        else:
            response = f"Nieznane polecenie: {cmd}"

        s.sendto(response.encode(), addr)
        print(f"[ODPOWIEDŹ -> {addr}] {response}")


## 8️⃣ Klient UDP — wysyła i odbiera odpowiedzi

In [ ]:

import socket

HOST = "127.0.0.1"
PORT = 6000

with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
    s.settimeout(2.0)
    print(f"Połączono z {HOST}:{PORT} (UDP)")
    print("Wpisz polecenie (echo, time, date, random) lub 'exit' aby zakończyć.")

    while True:
        message = input("> ")
        if message.lower() in ("exit", "quit"):
            print("Zakończono.")
            break

        s.sendto(message.encode(), (HOST, PORT))
        try:
            data, _ = s.recvfrom(1024)
            print("Odpowiedź:", data.decode())
        except socket.timeout:
            print("Brak odpowiedzi od serwera (timeout).")
